#### Importing Library Dependencies

In [ ]:
# Import
import contextily as ctx
import folium
import geodatasets
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
from owslib.ogcapi.features import Features
import pandas as pd
import requests

#### To check PyGeoAPI connectivity from the notebook environment

In [ ]:
# To check whether the pygeoapi server is running
# Endpoint should be following the url signature image name : port number as per the docker networking strategy
endpoint = "http://pygeoapi:80"
response = requests.get(endpoint)
if response.status_code == 200:
    print("pygeoapi server is running!")
else:
    print("pygeoapi server is not running.")


pygeoapi server is running!


#### To list the collections available in the server instance

In [ ]:
feature_dataset = Features('http://pygeoapi:80')
collections = feature_dataset.collections() 
print(json.dumps(collections, indent=4))

#### Access a specific feature of interest

In [ ]:

protected_sites = feature_dataset.collection('NSG')
print(json.dumps(protected_sites, indent=2))

#### View the queryable parameters and their datatypes

In [ ]:
# View the queryable parameters available in the dataset.
protected_sites_queryables = feature_dataset.collection_queryables('NSG')
print(json.dumps(protected_sites_queryables, indent=2))

#### Access items from concerned collection

In [ ]:
protected_sites_items = feature_dataset.collection_items('NSG') 
print(json.dumps(protected_sites_items, indent=4))

#### Store the queried feature items

In [ ]:
# Store the queried feature items
protected_sites_features = protected_sites_items['features']

# To access a feature from the stored variable
protected_sites_features[0]

#### To access items of a dataset from the PyGeoAPI endpoint

In [ ]:

# Specify the URL of your PyGeoAPI feature dataset
pygeoapi_url_to_access_dataset = "http://pygeoapi:80/collections/NSG/items"

# Make an HTTP GET request to fetch the GeoJSON data
response = requests.get(pygeoapi_url_to_access_dataset)
nsg_json_data = response.json()
print(nsg_json_data)

In [ ]:
# Create a GeoDataFrame directly from the GeoJSON data
nsg_data = gpd.GeoDataFrame.from_features(nsg_json_data['features'])
print(nsg_data)

In [ ]:
# Create a plot
ax = nsg_data.plot(figsize=(8, 8), alpha=0.5, edgecolor='k')

# Add basemap from OpenStreetMap using contextily with a valid zoom level 
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=10)

# Set the title
plt.title('Geometries on Base Map')

# Show the plot
plt.show()

In [ ]:
# Set the CRS for your GeoDataFrame (assuming it's EPSG 4647)
nsg_data.crs = 'EPSG:4647'

# Create a base map centered around the geometries
map_center = nsg_data.unary_union.centroid
m = folium.Map(location=[map_center.y, map_center.x], zoom_start=9)

# Style function to make the geometries deep blue
def style_function(feature):
    return {
        'fillColor': 'deepskyblue',  # Fill color
        'color': 'deepskyblue',      # Border color
        'weight': 2,                # Border width
        'fillOpacity': 0.6          # Fill opacity
    }

# Iterate through the GeoDataFrame and add geometries to the map
for index, row in nsg_data.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=style_function
    ).add_to(m)

# Save the map to an HTML file or display it in a Jupyter Notebook
# m.save('interactive_map.html')  # Save as an HTML file
m 

#### Visualizing the geometries with buffer regions

In [ ]:
nsg_data.crs = 'EPSG:4647'

# Create a base map centered around the geometries
map_center = nsg_data.unary_union.centroid
m = folium.Map(location=[map_center.y, map_center.x], zoom_start=9)

# Style function for the original geometries
def style_function_geom(feature):
    return {
        'fillColor': 'deepskyblue',  # Fill color for geometries
        'color': 'deepskyblue',      # Border color for geometries
        'weight': 2,                # Border width for geometries
        'fillOpacity': 0.6          # Fill opacity for geometries
    }

# Style function for the buffer areas
def style_function_buffer(feature):
    return {
        'fillColor': 'red',         # Fill color for buffer areas
        'color': 'red',             # Border color for buffer areas
        'weight': 2,                # Border width for buffer areas
        'fillOpacity': 0.6          # Fill opacity for buffer areas
    }

# Buffer function to create a buffer around the geometries
def create_buffer(geometry, buffer_distance):
    return geometry.buffer(buffer_distance)

# Iterate through the GeoDataFrame, add buffer areas, and add them to the map
buffer_distance = 0.02  # Adjust the buffer distance as needed
for index, row in nsg_data.iterrows():
    buffered_geometry = create_buffer(row['geometry'], buffer_distance)
    
    # Add the original geometries with style
    folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': row['geometry'].__geo_interface__,
            'properties': {},
        },
        style_function=style_function_geom
    ).add_to(m)

    # Add the buffered geometries with style
    folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': buffered_geometry.__geo_interface__,
            'properties': {},
        },
        style_function=style_function_buffer
    ).add_to(m)

# Save the map to an HTML file or display it in a Jupyter Notebook
# m.save('interactive_map_with_buffer.html')  # Save as an HTML file
m